## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
import os, sys
sys.path.insert(0, '..')

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zhigansk,RU,66.7697,123.3711,33.19,98,100,10.04,light snow
1,1,Sisimiut,GL,66.9395,-53.6735,41.95,60,4,5.46,clear sky
2,2,Chokurdakh,RU,70.6333,147.9167,-15.03,98,15,3.85,few clouds
3,3,Diego De Almagro,CL,-26.3667,-70.0500,72.66,28,0,10.83,clear sky
4,4,Vila Franca Do Campo,PT,37.7167,-25.4333,59.00,88,75,17.27,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Cabo San Lucas,MX,22.8909,-109.9124,80.60,41,1,8.05,clear sky
6,6,Rikitea,PF,-23.1203,-134.9692,78.58,70,99,16.15,overcast clouds
19,19,Kirakira,SB,-10.4544,161.9205,78.82,82,99,1.68,overcast clouds
28,28,Mahebourg,MU,-20.4081,57.7000,78.80,89,75,14.97,light rain
29,29,Kapaa,US,22.0752,-159.3190,75.00,73,75,17.27,broken clouds
...,...,...,...,...,...,...,...,...,...,...
694,694,Safidon,IN,29.4167,76.6667,76.80,19,0,7.11,clear sky
696,696,Guhagar,IN,17.4842,73.1929,79.07,73,3,8.34,clear sky
699,699,Payo,ID,-3.7508,103.6372,75.11,88,100,1.90,overcast clouds
702,702,Buraydah,SA,26.3260,43.9750,89.60,17,20,5.75,few clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                212
City                   212
Country                211
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Cabo San Lucas,MX,22.8909,-109.9124,80.60,41,1,8.05,clear sky
6,6,Rikitea,PF,-23.1203,-134.9692,78.58,70,99,16.15,overcast clouds
19,19,Kirakira,SB,-10.4544,161.9205,78.82,82,99,1.68,overcast clouds
28,28,Mahebourg,MU,-20.4081,57.7000,78.80,89,75,14.97,light rain
29,29,Kapaa,US,22.0752,-159.3190,75.00,73,75,17.27,broken clouds


In [16]:
clean_travel_cities.count()

City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Cabo San Lucas,MX,80.60,clear sky,22.8909,-109.9124,
6,Rikitea,PF,78.58,overcast clouds,-23.1203,-134.9692,
19,Kirakira,SB,78.82,overcast clouds,-10.4544,161.9205,
28,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,
29,Kapaa,US,75.00,broken clouds,22.0752,-159.3190,
35,Manicore,BR,88.07,broken clouds,-5.8092,-61.3003,
41,Coahuayana,MX,84.20,scattered clouds,18.7333,-103.6833,
46,Martapura,ID,78.80,overcast clouds,-3.4167,114.8500,
50,Souillac,MU,78.80,light rain,-20.5167,57.5167,
54,Tahta,EG,86.00,clear sky,26.7693,31.5021,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

hotel_df.head(25)

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Cabo San Lucas,MX,80.60,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
6,Rikitea,PF,78.58,overcast clouds,-23.1203,-134.9692,Pension Maro'i
19,Kirakira,SB,78.82,overcast clouds,-10.4544,161.9205,SanBiz Lodge
28,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
29,Kapaa,US,75.00,broken clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
35,Manicore,BR,88.07,broken clouds,-5.8092,-61.3003,Hotel Durval
41,Coahuayana,MX,84.20,scattered clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
46,Martapura,ID,78.80,overcast clouds,-3.4167,114.8500,Montana Hotel Syariah
50,Souillac,MU,78.80,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
54,Tahta,EG,86.00,clear sky,26.7693,31.5021,El Salam Hotel


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= ' ']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Cabo San Lucas,MX,80.60,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
6,Rikitea,PF,78.58,overcast clouds,-23.1203,-134.9692,Pension Maro'i
19,Kirakira,SB,78.82,overcast clouds,-10.4544,161.9205,SanBiz Lodge
28,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
29,Kapaa,US,75.00,broken clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
35,Manicore,BR,88.07,broken clouds,-5.8092,-61.3003,Hotel Durval
41,Coahuayana,MX,84.20,scattered clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
46,Martapura,ID,78.80,overcast clouds,-3.4167,114.8500,Montana Hotel Syariah
50,Souillac,MU,78.80,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
54,Tahta,EG,86.00,clear sky,26.7693,31.5021,El Salam Hotel


In [26]:
clean_hotel_df.count()

City                   211
Country                211
Max Temp               211
Current Description    211
Lat                    211
Lng                    211
Hotel Name             211
dtype: int64

In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))